# Amin1 cloud data access

This page provides information about how to access data from the [Amin1 resource](intro) via Google Cloud. This includes sample metadata and single nucleotide polymorphism (SNP) calls.

This notebook illustrates how to read data directly from the cloud, without having to first download any data locally. This notebook can be run from any computer, but will work best when run from a compute node within Google Cloud, because it will be physically closer to the data and so data transfer is faster. For example, this notebook can be run via [Google Colab](https://colab.research.google.com/) which is free interactive computing service running in the cloud.

To launch this notebook in the cloud and run it for yourself, click the launch icon (<i class="fas fa-rocket"></i>) at the top of the page and select one of the cloud computing services available.

## Data hosting

All data required for this notebook is hosted on Google Cloud Storage (GCS). Data are hosted in the `vo_amin_release` bucket, which is a multi-region bucket located in the United States. All data hosted in GCS are publicly accessible and do not require any authentication to access. 

## Setup

Running this notebook requires some Python packages to be installed. These packages can be installed via pip or conda. E.g.:

In [ ]:
!pip install -q malariagen_data

To make accessing these data more convenient, we've created the [malariagen_data Python package](https://github.com/malariagen/malariagen-data-python), which is available from PyPI. This is experimental so please let us know if you find any bugs or have any suggestions. 

Now import the packages we'll need to use here.

In [1]:
import malariagen_data

Data access from Google Cloud is set up with the following code:

In [2]:
amin1 = malariagen_data.Amin1("gs://vo_amin_release/")

## Sample metadata

Data about the samples that were sequenced to generate this data resource are available, including the time and place of collection, the gender of the specimen, and our call regarding the species of the specimen. These are organised by sample set.

Load sample metadata into a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe):

In [3]:
df_samples = amin1.sample_metadata()
df_samples

,sample_id,original_sample_id,sanger_sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,season,PCA_cohort,cohort,subsampled_cohort
0,VBS09378-4248STDY7308980,VBS09378,4248STDY7308980,CB-2-00264,Brandy St. Laurent,Cambodia,Preah Kleang,2016,3,13.667,104.982,Feb-Apr (late dry),A,PV,NaN
1,VBS09382-4248STDY7308981,VBS09382,4248STDY7308981,CB-2-00258,Brandy St. Laurent,Cambodia,Preah Kleang,2016,3,13.667,104.982,Feb-Apr (late dry),A,PV,NaN
2,VBS09397-4248STDY7308982,VBS09397,4248STDY7308982,CB-2-00384,Brandy St. Laurent,Cambodia,Preah Kleang,2016,3,13.667,104.982,Feb-Apr (late dry),A,PV,PV
3,VBS09460-4248STDY7308986,VBS09460,4248STDY7308986,CB-2-02960,Brandy St. Laurent,Cambodia,Preah Kleang,2016,6,13.667,104.982,May-Jul (early wet),A,PV,NaN
4,VBS09466-4248STDY7308989,VBS09466,4248STDY7308989,CB-2-04070,Brandy St. Laurent,Cambodia,Preah Kleang,2016,11,13.667,104.982,Nov-Jan (early dry),A,PV,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,VBS16624-4248STDY7918667,VBS16624,4248STDY7918667,KV-32-01591,Brandy St. Laurent,Cambodia,Sayas,2014,6,13.548,107.025,May-Jul (early wet),C,RK2,RK2
298,VBS16625-4248STDY7918668,VBS16625,4248STDY7918668,KV-32-01499,Brandy St. Laurent,Cambodia,Sayas,2014,6,13.548,107.025,May-Jul (early wet),C,RK2,RK2
299,VBS16626-4248STDY7918669,VBS16626,4248STDY7918669,KV-32-01465,Brandy St. Laurent,Cambodia,Sayas,2014,6,13.548,107.025,May-Jul (early wet),B,RK1,RK1
300,VBS16628-4248STDY7918670,VBS16628,4248STDY7918670,KV-32-01454,Brandy St. Laurent,Cambodia,Sayas,2014,6,13.548,107.025,May-Jul (early wet),C,RK2,RK2


The `sample_id` column gives the sample identifier used throughout all analyses.

The `country`, `location`, `latitude` and `longitude` columns give the location where the specimen was collected.

The `year` and `month` columns give the approximate date when the specimen was collected.

[Pandas](https://pandas.pydata.org/) can be used to explore and query the sample metadata in various ways. E.g., here is a summary of the numbers of samples by species:

In [4]:
df_summary = df_samples.pivot_table(
    index=["longitude", "latitude", "location"], 
    columns=["year"],
    values="sample_id", 
    aggfunc=len,
    fill_value=0
)
df_summary.style.set_caption("Number of mosquito specimens by collection site and year.")

,,year,2010,2011,2014,2015,2016
longitude,latitude,location,,,,,
102.735,12.155,Thmar Da,26,15,0,0,0
104.92,13.77,Chean Mok,0,0,66,9,0
104.982,13.667,Preah Kleang,0,0,47,9,36
106.995,13.595,Chamkar San,0,0,40,11,0
107.025,13.548,Sayas,0,0,39,4,0


## Reference genome

Sequence data in this study were aligned to the MINIMUS1 reference genome. This reference genome contains 678 contigs in total, but many contigs are small and not suitable for population genetic analyses. We therefore have included only SNP calls for the 40 largest contigs. The set of contigs analysed can be accessed as follows:

In [9]:
amin1.contigs

('KB663610',
 'KB663611',
 'KB663622',
 'KB663633',
 'KB663644',
 'KB663655',
 'KB663666',
 'KB663677',
 'KB663688',
 'KB663699',
 'KB663710',
 'KB663721',
 'KB663722',
 'KB663733',
 'KB663744',
 'KB663755',
 'KB663766',
 'KB663777',
 'KB663788',
 'KB663799',
 'KB663810',
 'KB663821',
 'KB663832',
 'KB663833',
 'KB663844',
 'KB663855',
 'KB663866',
 'KB663877',
 'KB663888',
 'KB663899',
 'KB663910',
 'KB663921',
 'KB663932',
 'KB663943',
 'KB663955',
 'KB664054',
 'KB664165',
 'KB664255',
 'KB664266',
 'KB664277')

For convenience, the reference genome sequence for any contig can be loaded as a [NumPy array](https://numpy.org/doc/stable/user/absolute_beginners.html), e.g.:

In [11]:
# load the reference sequence for a single contig as a numpy array
seq = amin1.genome_sequence(contig="KB663610").compute()
seq

array([b'T', b'T', b'C', ..., b'C', b'A', b'C'], dtype='|S1')

In [12]:
len(seq)

31626230

## SNP calls

We have called SNP genotypes in all samples at all positions in the genome where the reference allele is not "N". Data on the SNP positions, alleles, site filters and genotype calls for a given contig can be accessed as an [xarray Dataset](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset). E.g., access SNP calls for contig KB663610: 

In [13]:
df_snps = amin1.snp_calls(contig="KB663610")
df_snps

<xarray.Dataset>
Dimensions:              (variants: 22857027, alleles: 4, samples: 302, ploidy: 2)
Coordinates:
    variant_position     (variants) int32 dask.array<chunksize=(491338,), meta=np.ndarray>
    variant_contig       (variants) uint8 dask.array<chunksize=(491338,), meta=np.ndarray>
    sample_id            (samples) |S24 dask.array<chunksize=(302,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele       (variants, alleles) |S1 dask.array<chunksize=(491338, 1), meta=np.ndarray>
    variant_filter_pass  (variants) bool dask.array<chunksize=(886654,), meta=np.ndarray>
    call_genotype        (variants, samples, ploidy) int8 dask.array<chunksize=(277733, 50, 2), meta=np.ndarray>
    call_GQ              (variants, samples) int8 dask.array<chunksize=(277733, 50), meta=np.ndarray>
    call_MQ              (variants, samples) float32 dask.array<chunksize=(277733, 50), meta=np.ndarray>
    call_AD              (variants, samples, alleles) int16 dask.array<chunksize=(277733, 50, 4), meta=np.ndarray>
    call_genotype_mask   (variants, samples, ploidy) bool dask.array<chunksize=(277733, 50, 2), meta=np.ndarray>
Attributes:
    contigs:  ('KB663610', 'KB663611', 'KB663622', 'KB663633', 'KB663644', 'K...

The arrays within this dataset are backed by [Dask arrays](https://docs.dask.org/en/stable/array.html), and can be accessed as shown below.

### SNP positions and alleles

In [14]:
# SNP positions (1-based)
pos = df_snps['variant_position'].data
pos

dask.array<getitem, shape=(22857027,), dtype=int32, chunksize=(495697,), chunktype=numpy.ndarray>

In [15]:
# read first 10 SNP positions into a numpy array
p = pos[:10].compute()
p

array([209, 210, 212, 214, 215, 217, 219, 220, 221, 222], dtype=int32)

In [16]:
# SNP alleles (first column is the reference allele)
alleles = df_snps['variant_allele'].data
alleles

dask.array<getitem_variadic, shape=(22857027, 4), dtype=|S1, chunksize=(495697, 3), chunktype=numpy.ndarray>

In [18]:
# read first 10 SNP alleles into a numpy array
a = alleles[:10, :].compute()
a

array([[b'A', b'C', b'G', b'T'],
       [b'T', b'C', b'A', b'G'],
       [b'T', b'C', b'A', b'G'],
       [b'T', b'C', b'A', b'G'],
       [b'A', b'C', b'G', b'T'],
       [b'T', b'C', b'A', b'G'],
       [b'T', b'C', b'A', b'G'],
       [b'T', b'C', b'A', b'G'],
       [b'T', b'C', b'A', b'G'],
       [b'A', b'C', b'G', b'T']], dtype='|S1')

### Site filters

SNP calling is not always reliable, and we have created site filters to allow excluding low quality SNPs. For each contig, a "filter_pass" Boolean mask is available, where `True` indicates that the site passed the filter and is accessible to high quality SNP calling. 

In [19]:
# site filters
filter_pass = df_snps['variant_filter_pass'].data
filter_pass

dask.array<getitem, shape=(22857027,), dtype=bool, chunksize=(886654,), chunktype=numpy.ndarray>

In [21]:
# load site filters for first 10 SNPs
f = filter_pass[:10].compute()
f

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [24]:
# how many sites on this contig pass filters?
n_sites = filter_pass.shape[0]
n_pass = filter_pass.sum().compute()
print(f"{n_pass:,} out of {n_sites:,} ({n_pass/n_sites:.0%}) sites pass site filters")

22,857,027 out of 22,857,027 (100%) sites pass site filters


In [25]:
for contig in amin1.contigs:
    df_snps = amin1.snp_calls(contig=contig)
    filter_pass = df_snps['variant_filter_pass'].data
    n_sites = filter_pass.shape[0]
    n_pass = filter_pass.sum().compute()
    print(f"{contig}: {n_pass:,} out of {n_sites:,} ({n_pass/n_sites:.0%}) sites pass site filters")

KB663610: 22,857,027 out of 22,857,027 (100%) sites pass site filters
KB663611: 3,737,822 out of 3,737,822 (100%) sites pass site filters
KB663622: 4,388,762 out of 4,388,762 (100%) sites pass site filters
KB663633: 4,215,569 out of 4,215,569 (100%) sites pass site filters
KB663644: 4,221,032 out of 4,221,032 (100%) sites pass site filters
KB663655: 2,916,639 out of 2,916,639 (100%) sites pass site filters
KB663666: 2,501,655 out of 2,501,655 (100%) sites pass site filters
KB663677: 2,586,150 out of 2,586,150 (100%) sites pass site filters
KB663688: 2,742,023 out of 2,742,023 (100%) sites pass site filters
KB663699: 1,534,228 out of 1,534,228 (100%) sites pass site filters
KB663710: 1,213,646 out of 1,213,646 (100%) sites pass site filters
KB663721: 15,452,041 out of 15,452,041 (100%) sites pass site filters
KB663722: 1,209,792 out of 1,209,792 (100%) sites pass site filters
KB663733: 1,062,674 out of 1,062,674 (100%) sites pass site filters
KB663744: 1,240,722 out of 1,240,722 (100%) 

Note that we have chosen to genotype all samples at all sites in the genome, assuming all possible SNP alleles. Not all of these alternate alleles will actually have been observed in the samples. To determine which sites and alleles are segregating, an allele count can be performed over the samples you are interested in. See the example below. 

### SNP genotypes

SNP genotypes for individual samples are available. Genotypes are stored as a three-dimensional array, where the first dimension corresponds to genomic positions, the second dimension is samples, and the third dimension is ploidy (2). Values coded as integers, where -1 represents a missing value, 0 represents the reference allele, and 1, 2, and 3 represent alternate alleles.

SNP genotypes can be accessed as dask arrays as shown below.

## Example computation

@@TODO

## Feedback and suggestions

If there are particular analyses you would like to run, or if you have other suggestions for useful documentation we could add to this site, we would love to know, please get in touch via the [malariagen/vector-data GitHub discussion board](https://github.com/malariagen/vector-data/discussions).

## API docs

Here are the docstrings for the functions in the `malariagen_data` package that we used above.

In [26]:
help(amin1.sample_metadata)

Help on method sample_metadata in module malariagen_data.amin1:

sample_metadata() method of malariagen_data.amin1.Amin1 instance
    Access sample metadata.
    
    Returns
    -------
    df : pandas.DataFrame



In [28]:
help(amin1.genome_sequence)

Help on method genome_sequence in module malariagen_data.amin1:

genome_sequence(contig, inline_array=True, chunks='native') method of malariagen_data.amin1.Amin1 instance
    Access the reference genome sequence.
    
    Parameters
    ----------
    contig : str
        Chromosome arm, e.g., "3R".
    inline_array : bool, optional
        Passed through to dask.array.from_array().
    chunks : str, optional
        If 'auto' let dask decide chunk size. If 'native' use native zarr chunks.
        Also can be a target size, e.g., '200 MiB'.
    
    Returns
    -------
    d : dask.array.Array



In [29]:
help(amin1.snp_calls)

Help on method snp_calls in module malariagen_data.amin1:

snp_calls(contig, site_mask=False, inline_array=True, chunks='native') method of malariagen_data.amin1.Amin1 instance
    Access SNP sites, site filters and genotype calls.
    
    Parameters
    ----------
    contig : str or list
        Contig, e.g., "KB663610", or list of contigs.
    site_mask : bool
        Apply site filters.
    inline_array : bool, optional
        Passed through to dask.array.from_array().
    chunks : str, optional
        If 'auto' let dask decide chunk size. If 'native' use native zarr chunks.
        Also can be a target size, e.g., '200 MiB'.
    
    Returns
    -------
    ds : xarray.Dataset



---